In [72]:
import pandas as pd
import lxml.etree 
import csv
import tarfile
import os
from tqdm import tqdm

import glob

In [75]:


def getHeaderFromXSLT(xslt):
    root=xslt.getroot()
    
    headers=[]

    for item in root.findall('.//xsl:value-of',namespaces=root.nsmap):
        #ignore variables
        if (item.find('..').tag!='{http://www.w3.org/1999/XSL/Transform}variable'):
            el = item.attrib['select'].replace('.//','').replace('@','').replace('normalize-space(translate(','').replace(",'[¬]',''))",'').replace('$','').replace('/','__')
            if(el=='.'):
                #select the parent of the tag in order ot get select attribute
                el = item.find('..').attrib['select'].replace('.//','').replace('@','').replace('normalize-space(translate(','').replace(",'[¬]',''))",'').replace('$','').replace('/','__')
            headers.append(el)
    return headers


    



def readTar(file:str):
    with tarfile.open(file, "r:gz") as tar:
        # Go over each member
        for member in tar:
            if '.xml' in member.name:
                yield tar.extractfile(member).read()  

def writeRows(writer,tree,header,xslt):
    transform = lxml.etree.XSLT(xslt)
    transformed=transform(tree)
    lines=str(transformed).split('$end_line$')
    for line in lines:
        #write only if there are values in the line
        if (len(header)<=len(line)):
            writer.writerow(line.split('¬'))

# open your tar.gz file

file='ORCID_2021_10_summaries.tar.gz'


outdir='data'
os.makedirs(outdir, exist_ok=True)

xslt_files=[]
headers=[]
files=[]
filenames=[]
writers=[]

outdir='data'

for xslt_file in glob.glob('xslt/*.xsl'):
    xslt=lxml.etree.parse(xslt_file)
    xslt_files.append(xslt)
    name=os.path.basename(xslt_file).replace('.xsl','')
    filenames.append(name)
    f=open(os.path.join(outdir, "orcid_"+name+'.csv'), "w", encoding="utf8")
    files.append(f)
    headers.append(getHeaderFromXSLT(xslt))
    writers.append(csv.writer(f, lineterminator="\n"))


#write header row
for i in range(0,len(files)):
    writers[i].writerow(headers[i])


import itertools

for xml in tqdm(itertools.islice(readTar(file), 10000),total=10000):
#for xml in tqdm(readTar(file),total=12000000):
    # Extract member
    tree = lxml.etree.fromstring(xml)
    for i in range(0,len(files)):
        writeRows(writers[i],tree,headers[i],xslt_files[i])
        

for file in files:
    file.close()
        


100%|██████████| 10000/10000 [00:35<00:00, 278.04it/s]


In [61]:

# open your tar.gz file

file='ORCID_2021_10_summaries.tar.gz'

def getHeaderFromXSLT(xslt):
    root=xslt.getroot()
    
    headers=[]

    for item in root.findall('.//xsl:value-of',namespaces=root.nsmap):
        #ignore variables
        if (item.find('..').tag!='{http://www.w3.org/1999/XSL/Transform}variable'):
            el = item.attrib['select'].replace('.//','').replace('@','').replace('normalize-space(translate(','').replace(",'[¬]',''))",'').replace('$','').replace('/','__')
            if(el=='.'):
                #select the parent of the tag in order ot get select attribute
                el = item.find('..').attrib['select'].replace('.//','').replace('@','').replace('normalize-space(translate(','').replace(",'[¬]',''))",'').replace('$','').replace('/','__')
            headers.append(el)
    return headers


    



def readTar(file:str):
    with tarfile.open(file, "r:gz") as tar:
        # Go over each member
        for member in tar:
            if '.xml' in member.name:
                yield tar.extractfile(member).read()  

def writeRows(writer,tree,header,xslt):
    transform = lxml.etree.XSLT(xslt)
    transformed=transform(tree)
    lines=str(transformed).split('$end_line$')
    for line in lines:
        #write only if there are values in the line
        if (len(header)!=len(line)):
            writer.writerow(line.split('¬'))

outdir='data'
os.makedirs(outdir, exist_ok=True)




with open(
    os.path.join(outdir, "orcid_profiles.csv"), "w", encoding="utf8") as op, open(
        os.path.join(outdir, "orcid_external-identifiers.csv"), "w", encoding="utf8") as oei, open(
            os.path.join(outdir, "orcid_researcher-url.csv"), "w", encoding="utf8") as oru, open(
                os.path.join(outdir, "orcid_distinctions.csv"), "w", encoding="utf8") as od:
        
    op_writer = csv.writer(op, lineterminator="\n")
    oei_writer = csv.writer(oei, lineterminator="\n")
    oru_writer = csv.writer(oru, lineterminator="\n")
    od_writer = csv.writer(od, lineterminator="\n")

    #load XSLT defintions
    xslt_profile=lxml.etree.parse('xslt/profile.xsl')
    xslt_external_identifier=lxml.etree.parse('xslt/external-identifier.xsl')
    xslt_researcher_url=lxml.etree.parse('xslt/researcher-url.xsl')
    xslt_distinctions=lxml.etree.parse('xslt/distinctions.xsl')

    #get CSV headers from XSLT
    header_profile =getHeaderFromXSLT(xslt_profile)
    header_identifiers=getHeaderFromXSLT(xslt_external_identifier)
    header_researcher_url=getHeaderFromXSLT(xslt_researcher_url)
    header_distinctions=getHeaderFromXSLT(xslt_distinctions)

    #write header row
    op_writer.writerow(header_profile)
    oei_writer.writerow(header_identifiers)
    oru_writer.writerow(header_researcher_url)
    od_writer.writerow(header_distinctions)
    import itertools

    for xml in tqdm(itertools.islice(readTar(file), 10000),total=10000):
    #for xml in tqdm(readTar(file),total=12000000):
        # Extract member
        tree = lxml.etree.fromstring(xml)
        writeRows(op_writer,tree,header_profile,xslt_profile)
        writeRows(oei_writer,tree,header_identifiers,xslt_external_identifier)
        writeRows(oru_writer,tree,header_researcher_url,xslt_researcher_url)
        writeRows(od_writer,tree,header_distinctions,xslt_distinctions)
        
            
    

OSError: Error reading file 'xslt/profile.xsl': failed to load external entity "xslt/profile.xsl"